In [ ]:
import pygplates
import numpy as np
import pandas as pd

In [ ]:
Laurentia_poles = pd.read_

In [ ]:
def make_GPML(pmag_data,output_file_name):
    vgps = []
    for n in pmag_data.index:
        # The pole position and the average sample site position.
        pole_position = pygplates.PointOnSphere(pmag_data.PLAT[n], pmag_data.PLONG[n])
        average_sample_site_position = pygplates.PointOnSphere(pmag_data.SLAT[n], pmag_data.SLONG[n])

        # Create the virtual geomagnetic pole feature.
        virtual_geomagnetic_pole_feature = pygplates.Feature.create_reconstructable_feature(
            pygplates.FeatureType.gpml_virtual_geomagnetic_pole,pole_position,
            name=str(pmag_data.ROCKNAME[n]), 
            reconstruction_plate_id=pmag_data.Plate_ID[n])
    #reconstruction_plate_id=701

In [ ]:
def make_GPML_w_plates(pmag_data,static_polygons_file_path,
                       rotation_model_file_path,output_file_name):
    """
    pmag_data : data frame with paleomagnetic poles. It needs to have fields
                named PLONG, PLAT, SLONG, SLAT, A95 and nominal_age
    static_polygons_file_path : string to static polygons are the 'partitioning 
        features used to assign plate IDs
    rotation_model_file_path : string to rotation model partition_into_plates 
        function requires a rotation model, since sometimes this would be 
        necessary even at present day (for example to resolve topological polygons)
    output_file_name : string for output file
    """
    vgps = []
    for n in pmag_data.index:
        # The pole position and the average sample site position.
        pole_position = pygplates.PointOnSphere(pmag_data.PLAT[n], pmag_data.PLONG[n])
        average_sample_site_position = pygplates.PointOnSphere(pmag_data.SLAT[n], pmag_data.SLONG[n])

        # Create the virtual geomagnetic pole feature.
        virtual_geomagnetic_pole_feature = pygplates.Feature.create_reconstructable_feature(
            pygplates.FeatureType.gpml_virtual_geomagnetic_pole,pole_position,name=str(pmag_data.ROCKNAME[n]))
    #reconstruction_plate_id=701

        # Set the average sample site position.
        # We need to specify its property name otherwise it defaults to the pole position and overwrites it.
        virtual_geomagnetic_pole_feature.set_geometry(
            average_sample_site_position,
            pygplates.PropertyName.gpml_average_sample_site_position)

        # Set the pole position uncertainty and the average age.
        virtual_geomagnetic_pole_feature.set_double(
            pygplates.PropertyName.gpml_pole_a95,
            float(pmag_data.A95[n]))
        virtual_geomagnetic_pole_feature.set_double(
            pygplates.PropertyName.gpml_average_age,
            pmag_data['nominal age'][n])
        vgps.append(virtual_geomagnetic_pole_feature)
        
    static_polygons = pygplates.FeatureCollection(static_polygons_file_path)
    rotation_model = pygplates.RotationModel(rotation_model_file_path)
        
    # The partition points function can then be used as before
    vgps_plates = pygplates.partition_into_plates(static_polygons,rotation_model,vgps)

    output_features = pygplates.FeatureCollection(vgps_plates)

    # Note that the output format could be shp, gmt, gpml or gpmlz - the extension controls the format
    output_features.write(output_file_name)